https://github.com/robinvanschaik/interpret-flair

In [1]:
import pandas as pd
import datetime
from flair.models import TextClassifier
from flair.data import Sentence
import torch
import torch.nn as nn
from transformers import AutoTokenizer
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization
from interpretation_package.flair_model_wrapper import ModelWrapper
from interpretation_package.interpret_flair import interpret_sentence, visualize_attributions
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
pd.set_option('display.max_rows', 200)

In [3]:
classifier = TextClassifier.load('sentiment')

2021-02-25 23:51:08,826 loading file /media/joey/Samsung_T5/FLAIR_CACHE_ROOT/models/sentiment-en-mix-distillbert_3.1.pt


In [4]:
# redefine the function from interpret_flair, in order to control whether to norm the word_attributions before sum
# also prevent the attributions becomes Nan if the norm is 0.
def summarize_attributions(attributions, need_norm=True):
    """
    Helper function for calculating word attributions.
    Inputs:
    attributions_ig: integrated gradients attributions.
    Ouputs:
    word_attributions: the attributions score per token.
    attribution_score: the attribution score of the entire document w.r.t. ground label.
    """
    attributions = attributions.sum(dim=-1).squeeze(0)
    if need_norm==True:
        if torch.norm(attributions) != 0:
            attributions = attributions / torch.norm(attributions)
    attribution_score = attributions.sum()

    return attributions, attribution_score

In [5]:
# redefine the function from interpret_flair, take control of whether to add special tokens, 
#                      whether to add softmax layer in the attribution calculation or after
#                       whether to use all zero baseline or base_line filled with [PAD] token
def interpret_sentence2(flair_model_wrapper, lig, sentence, target_label, visualization_list, n_steps=100, estimation_method="gausslegendre",internal_batch_size=None, add_special_tokens=True, need_softmax = True, need_norm=True, pad_base_line=True):
    """
    We can visualise the attributions made by making use of Pytorch Captum.
    Inputs:
    flair_model_wrapper: class containing a customized forward function of Flair model.
    lig: the layer integrated gradient object.
    sentence: the Flair sentence-object we want to interpret.
    target_label: the ground truth class-label of the sentence.
    visualization_list: a list to store the visualization records in.
    """

    # Return the target index from the label dictionary.
    target_index = flair_model_wrapper.label_dictionary.get_idx_for_item(target_label)
#     target_index = target_index.to(device)
    
    # In order maintain consistency with Flair, we apply the same tokenization
    # steps.
    flair_sentence = Sentence(sentence)

    tokenized_sentence = flair_sentence.to_tokenized_string()

    tokenizer_max_length = flair_model_wrapper.tokenizer.model_max_length

    # This calculates the token input IDs tensor for the model.
    input_ids = flair_model_wrapper.tokenizer.encode(tokenized_sentence,
                                                     add_special_tokens=add_special_tokens,
                                                     max_length=tokenizer_max_length,
                                                     truncation=True,
                                                     return_tensors="pt")
    input_ids = input_ids.to(device)
    
    # Create a baseline by creating a tensor of equal length
    # containing the padding token tensor id.
    pad_token_id = flair_model_wrapper.tokenizer.pad_token_id

    ref_base_line = torch.full_like(input_ids, pad_token_id)

    # Convert back to tokens as the model requires.
    # As some words might get split up. e.g. Caroll to Carol l.
    all_tokens = flair_model_wrapper.tokenizer.convert_ids_to_tokens(input_ids[0])

    # The tokenizer in the model adds a special character
    # in front of every sentence.
    readable_tokens = [token.replace("▁", "") for token in all_tokens]

    # The input IDs are passed to the embedding layer of the model.
    # It is better to return the logits for Captum.
    # https://github.com/pytorch/captum/issues/355#issuecomment-619610044
    # Thus we calculate the softmax afterwards.
    # For now, I take the first dimension and run this sentence, per sentence.
    model_outputs = flair_model_wrapper(input_ids)
    if need_softmax == True:
        softmax = torch.nn.functional.softmax(model_outputs[0], dim=0)
        # Return the confidence and the class ID of the top predicted class.
        conf, idx = torch.max(softmax, 0)
    else:
        conf, idx = torch.max(model_outputs[0], 0)
    # Returns the probability.
    prediction_confidence = conf.item()

    # Returns the label name from the top prediction class.
    pred_label = flair_model_wrapper.label_dictionary.get_item_for_index(idx.item())
    if pad_base_line == True:
    # Calculate the attributions according to the LayerIntegratedGradients method.
        attributions_ig, delta = lig.attribute(input_ids,
                                               baselines=ref_base_line,
                                               n_steps=n_steps,
                                               return_convergence_delta=True,
                                               target=target_index,
                                               method=estimation_method,
                                               internal_batch_size=internal_batch_size)
    else:
        attributions_ig, delta = lig.attribute(input_ids,
                                               n_steps=n_steps,
                                               return_convergence_delta=True,
                                               target=target_index,
                                               method=estimation_method,
                                               internal_batch_size=internal_batch_size)
    convergence_delta = abs(delta)
    print('pred: ', idx.item(), '(', '%.2f' % conf.item(), ')', ', delta: ', convergence_delta)


    word_attributions, attribution_score = summarize_attributions(attributions_ig, need_norm=need_norm)


    visualization_list.append(
    visualization.VisualizationDataRecord(word_attributions=word_attributions,
                                pred_prob=prediction_confidence,
                                pred_class=pred_label,
                                true_class=target_label,
                                attr_class=target_label,
                                attr_score=attribution_score,
                                raw_input=readable_tokens,
                                convergence_score=delta)
                    )

    # Return these for the sanity checks.
    return readable_tokens, word_attributions, convergence_delta

In [6]:
# redefine the ModelWrapper, adding the softmax inside
class ModelWrapper2(nn.Module):

    def __init__(self, flair_model, layers: str = "-1"):
        super(ModelWrapper2, self).__init__()

        # Pass the flair
        self.flair_model = flair_model
        
        # Shorthand for the actual PyTorch model.
        self.model = flair_model.document_embeddings.model

        if torch.cuda.is_available():
            self.device = torch.device("cuda:0")
        else:
            self.device = torch.device("cpu")
            
        self.model.eval()
        self.model.zero_grad()

        # Split the name to automatically grab the right tokenizer.
        self.model_name = flair_model.document_embeddings.get_names()[0].split('transformer-document-')[-1]
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        self.label_dictionary = self.flair_model.label_dictionary
        self.num_classes = len(self.flair_model.label_dictionary)
        self.embedding_length = self.flair_model.document_embeddings.embedding_length

        self.initial_cls_token = flair_model.document_embeddings.initial_cls_token

        if layers == 'all':
            # send mini-token through to check how many layers the model has
            hidden_states = self.model(torch.tensor([1], device=device).unsqueeze(0))[-1]
            self.layer_indexes = [int(x) for x in range(len(hidden_states))]
        else:
            self.layer_indexes = [int(x) for x in layers.split(",")]
        
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids):
        # Run the input embeddings through all the layers.
        # Return the hidden states of the model.
        hidden_states = self.model(input_ids=input_ids)[-1]

        # BERT has an initial CLS token.
        # Meaning that the the first token contains the classification.
        # Other models have this as the top layer.
        index_of_CLS_token = 0 if self.initial_cls_token else input_ids.shape[1] -1

        # For batching we need to replace
        # [layer][0][index_of_CLS_token]
        # with [layer][i][index_of_CLS_token].
        cls_embeddings_all_layers = \
            [hidden_states[layer][0][index_of_CLS_token] for layer in self.layer_indexes]

        output_embeddings = torch.cat(cls_embeddings_all_layers)

        # https://github.com/pytorch/captum/issues/355#issuecomment-619610044
        # It's better to attribute the logits to the inputs.
        label_scores = self.flair_model.decoder(output_embeddings)

        # Captum expects [#examples, #classes] as size.
        # We do to this so we can specify the target class with multiclass
        # models.
        label_scores_resized = torch.reshape(label_scores, (1, self.num_classes))
        
        label_scores_softmaxed = self.softmax(label_scores_resized)
        return label_scores_softmaxed

In [7]:
flair_model_wrapper2 = ModelWrapper2(classifier)
lig2 = LayerIntegratedGradients(flair_model_wrapper2, flair_model_wrapper2.model.embeddings)

In [8]:
df = pd.read_csv('flair_analysis_0212.csv')
df

,Previous date,Voting date,Person1,Person2,Flair_score
0,9/11/13,9/25/13,Marks,Gillum,-2.2658
1,9/11/13,9/25/13,Marks,Miller,-2.2658
2,9/11/13,9/25/13,Gillum,Miller,-2.2658
3,9/11/13,9/25/13,Marks,Ziffer,-2.2658
4,9/11/13,9/25/13,Gillum,Ziffer,-2.2658
5,9/11/13,9/25/13,Miller,Ziffer,-2.2658
6,9/11/13,9/25/13,Marks,Maddox,-2.2658
7,9/11/13,9/25/13,Gillum,Maddox,-2.2658
8,9/11/13,9/25/13,Miller,Maddox,-2.2658
9,9/11/13,9/25/13,Ziffer,Maddox,-2.2658


In [9]:
groups = df.groupby(['Previous date', 'Voting date'])

In [10]:
groups.count()

,,Person1,Person2,Flair_score
Previous date,Voting date,,,
10/23/13,11/26/13,18,18,18
5/22/13,6/19/13,20,20,20
5/8/13,5/22/13,24,24,24
5/9/12,5/23/12,8,8,8
8/28/13,9/11/13,54,54,54
9/11/13,9/25/13,60,60,60


In [11]:
time_range = list(groups.groups.keys())

In [12]:
for date in groups.groups.keys():
    group = groups.get_group(date)
#     group = groups.get_group('9/11/13')
    people_set = set()
    for person1, person2 in zip(group['Person1'], group['Person2']):
        people_set.add(person1)
        people_set.add(person2)
    print(people_set)

{'Ziffer', 'Marks', 'Miller'}
{'Ziffer', 'Miller', 'Maddox', 'Marks', 'Gillum'}
{'Ziffer', 'Miller', 'Maddox', 'Marks', 'Gillum'}
{'Maddox', 'Gillum'}
{'Ziffer', 'Miller', 'Maddox', 'Marks', 'Gillum'}
{'Ziffer', 'Miller', 'Maddox', 'Marks', 'Gillum'}


In [26]:
group = groups.get_group('9/11/13')
people_set = set()
for person1, person2 in zip(group['Person1'], group['Person2']):
    people_set.add(person1)
    people_set.add(person2)
    print(people_set)

In [13]:
df_s1 = pd.read_csv('new_df_new_s1.csv', index_col=0)
df_s2 = pd.read_csv('new_df_new_s2.csv', index_col=0)
df_s3 = pd.read_csv('new_df_new_s3.csv', index_col=0)
df_s4 = pd.read_csv('new_df_new_s4.csv', index_col=0)

In [14]:
df_total = df_s1.append(df_s2).append(df_s3).append(df_s4)
df_total = df_total.reset_index(drop=True)
length = len(df_total)

body_list = df_total['Body'].values.tolist()
for i in range(length):
    body_list[i] = eval(body_list[i])
df_total['Body'] = body_list
sent_list = pd.to_datetime(df_total['Sent'])
df_total

,From,Sent,To,CC,Subject,Attachments,Importance,Body,textblob_sentiment,nltk_sentiment,flair_sentiment,GCP_sentiment,new_flair
0,"('scott', 'maddox', 'shamaddox@embarqmail.com')",2013-05-09 17:47:00,"[('scott', 'maddox', 'scott.maddox@talgov.com')]",[],[],[],[],"[, I have already booked the rooms for LF; Thu...","[0.20677083333333335, 0.49444444444444446]","[0.9484, 0.028, 0.783, 0.188]",POSITIVE (0.9996),"[0.10000000149011612, 2.0999999046325684, 0.01...",0.9992
1,"('scott', 'maddox', 'shamaddox@embarqmail.com')",2013-05-09 17:47:00,"[('scott', 'maddox', 'scott.maddox@talgov.com')]",[],[],[],[],[Re: FLORIDA LEAGUE OF CITIES: Nominating Comm...,"[0.23080357142857144, 0.530952380952381]","[0.9576, 0.025, 0.786, 0.188]",POSITIVE (0.9983),"[0.10000000149011612, 2.200000047683716, 0.016...",0.9966
2,"('scott', 'maddox', 'scott.maddox@talgov.com')",2013-05-30 19:08:00,"[('favors thompson, anita', '', 'anita.favors....","[('john', 'marks', 'john.marksiii@talgov.com')...",[],[],[],"[Re: recommendations for appointment, Great ch...","[0.8, 0.75]","[0.6249, 0.0, 0.549, 0.451]",POSITIVE (0.9831),"[0.10000000149011612, 0.10000000149011612, 0.0...",0.9662
3,"('john', 'marks', 'john.marksiii@talgov.com')",2013-11-25 15:03:00,"[('gil', 'ziffer', 'gil@ziffberry.com'), (',',...","[('f20009', 'l20009', 'mikesittig@flcities.com...",[],[],[],[RE: Board meeting My flight that same morning...,"[0.46875, 0.71875]","[0.6369, 0.026, 0.879, 0.094]",NEGATIVE (0.835),"[0.4000000059604645, 2.700000047683716, 0.0360...",-0.6700
4,"('scott', 'maddox', 'shamaddox@embarqmail.com')",2013-05-10 09:10:00,"[('scott', 'maddox', 'scott.maddox@talgov.com')]",[],[],[],[],[Re: FLORIDA LEAGUE OF CITIES: Nominating Comm...,"[0.28055555555555556, 0.34629629629629627]","[0.9393, 0.0, 0.878, 0.122]",POSITIVE (0.9929),"[0.0, 0.8999999761581421, 0.006249999834431542]",0.9858
5,"('gil', 'ziffer', 'gil.ziffer@talgov.com')",2013-03-13 20:38:00,"[('scott', 'maddox', 'scott.maddox@talgov.com')]",[],[],[],[],"[League dinner Tuesday night, Is this typicall...","[0.016666666666666677, 0.35]","[0.4404, 0.0, 0.805, 0.195]",POSITIVE (0.9179),"[0.20000000298023224, 0.5, 0.03333333333333333]",0.8358
6,"('scott', 'maddox', 'scott@scottmaddox.com')",2012-11-27 18:20:00,"[('paige', 'carter-smith', 'paige@govinc.net')]","[('sandra', 'manning', 'sandra_manning'), ('sc...",[],[],[],"[, But if you just give me the talking point y...","[0.5, 1.0]","[-0.2732, 0.144, 0.778, 0.078]",POSITIVE (0.7863),"[0.20000000298023224, 0.20000000298023224, 0.0...",0.5726
7,"('scott', 'maddox', 'scott.maddox@talgov.com')",2012-11-27 18:20:00,"[('paige', 'carter-smith', 'paige@govinc.net')]","[('harry', 'reed', 'sandra.manning@talgov.com'...",[],[],[],"[, But if you just give me the talking point y...","[0.5, 1.0]","[-0.2732, 0.144, 0.778, 0.078]",POSITIVE (0.7863),"[0.20000000298023224, 0.20000000298023224, 0.0...",0.5726
8,"('john', 'marks', 'john.marksiii@talgov.com')",2013-05-21 16:28:00,"[('favors thompson, anita', '', 'anita.favors....","[('andrew', 'gillum', 'andrew.gillum@talgov.co...",[],[],[],"[RE: Cultural Plan Report, Please keep me post...","[0.1, 0.1]","[0.3182, 0.0, 0.813, 0.187]",NEGATIVE (0.9341),"[0.10000000149011612, 0.20000000298023224, 0.0...",-0.8682
9,"('scott', 'maddox', 'scott.maddox@talgov.com')",2013-03-25 13:31:00,"[('f4395', 'l4395', 'scott@scottmaddox2012.com...",[],['Fwd: Emailing: CAONoiseOrdinance2-26-13.doc'],[''],[],"[CAONoiseOrdinance2-26-13.doc; ATT00001.htm, R...","[0.0, 0.0]","[0.0, 0.0, 1.0, 0.0]",NEGATIVE (0.9238),"[0.0, 0.0, 0.0]",-0.8476


In [15]:
target_people_list =[('john', 'marks', 'john.marks@talgov.com'), 
                     ('john', 'marks', 'john.marksiii@talgov.com'),
                     ('andrew', 'gillum', 'andrew@andrewgillum.com'),
                     ('commissionerandrew', 'gillum', 'commissionerandrew_gillum'),
                     ('andrew', 'gillum', 'ademetricg@gmail.com'),
                     ('andrew', 'gillum', 'gilluma@talgov.com'),
                     ('andrew', 'gillum', 'agillum@pfaw.org'),
                     ('andrew', 'gillum', 'andrew.gillum@talgov.com'),
                     ('mark', 'mustian', 'mmustian@ngn-tally.com'),
                     ('mark', 'mustian', 'mmustian@ngnlaw.com'),
                     ('jessica', 'miller', 'jessica.miller@talgov.com'),
                     ('gil', 'ziffer', 'gil.ziffer@talgov.com'),
                     ('gil', 'ziffer', 'gil@ziffberry.com'),
                     ('scott', 'maddox', 'maddox@maddoxhorne.com'),
                     ('scott', 'maddox', 'scott.maddox@talgov.com'),
                     ('sha', 'maddox', 'sha_maddox'),
                     ('scott', 'maddox', 'scott@scottmaddox.com'),
                     ('scott', 'maddox', 'shamaddox@embarqmail.com'),
                     ('scott', 'maddox', 'scottcharlesmaddox@gmail.com'),
                     ('curtis', 'richardson', 'cabaide@aol.com'),
                     ('ultis', 'richardson', 'ultis_richardson')
                    ]

In [16]:
idx_list = []
standard_time_range = []
for tup in time_range:
    temp_list=[]
    start_time = datetime.datetime.strptime(tup[0], '%m/%d/%y') 
    end_time = datetime.datetime.strptime(tup[1], '%m/%d/%y')   
    standard_time_range.append([start_time, end_time])
    print(start_time, end_time)
    for i in range(length):
        if sent_list[i] > start_time and sent_list[i] < end_time:
            print(i)
            temp_list.append(i)
    idx_list.append(temp_list)

2013-10-23 00:00:00 2013-11-26 00:00:00
3
2013-05-22 00:00:00 2013-06-19 00:00:00
2
17
35
37
46
71
72
2013-05-08 00:00:00 2013-05-22 00:00:00
0
1
4
8
12
48
53
54
57
58
59
60
62
63
64
73
74
75
76
2012-05-09 00:00:00 2012-05-23 00:00:00
27
2013-08-28 00:00:00 2013-09-11 00:00:00
36
50
51
2013-09-11 00:00:00 2013-09-25 00:00:00
39
55
56


# with 

In [17]:
for i in range(6):
    print(standard_time_range[i])
    df_temp = df_total.iloc[idx_list[i]]
    df_temp = df_temp.reset_index(drop = True)
    for sentence in df_temp['Body']:
        s = Sentence(sentence)
        classifier.predict(s)
#         print(s)
    visualization_list = []
    target_label = 'POSITIVE'
    for sentence in df_temp['Body']:
        interpret_sentence2(flair_model_wrapper2,
                            lig2,
                            sentence,
                            target_label,
                            visualization_list,
                            n_steps=500,
                            estimation_method="gausslegendre",
                            add_special_tokens=False, 
                            need_softmax = False,
                            internal_batch_size=3)
    visualize_attributions(visualization_list)

[datetime.datetime(2013, 10, 23, 0, 0), datetime.datetime(2013, 11, 26, 0, 0)]
pred:  0 ( 0.94 ) , delta:  tensor([0.5882], device='cuda:0', dtype=torch.float64)


[datetime.datetime(2013, 5, 22, 0, 0), datetime.datetime(2013, 6, 19, 0, 0)]
pred:  1 ( 0.86 ) , delta:  tensor([0.0677], device='cuda:0', dtype=torch.float64)
pred:  0 ( 0.96 ) , delta:  tensor([0.5977], device='cuda:0', dtype=torch.float64)
pred:  1 ( 1.00 ) , delta:  tensor([0.0504], device='cuda:0', dtype=torch.float64)
pred:  0 ( 0.70 ) , delta:  tensor([0.4342], device='cuda:0', dtype=torch.float64)
pred:  1 ( 0.89 ) , delta:  tensor([0.0419], device='cuda:0', dtype=torch.float64)
pred:  1 ( 0.59 ) , delta:  tensor([0.2651], device='cuda:0', dtype=torch.float64)
pred:  1 ( 0.89 ) , delta:  tensor([0.0396], device='cuda:0', dtype=torch.float64)


[datetime.datetime(2013, 5, 8, 0, 0), datetime.datetime(2013, 5, 22, 0, 0)]
pred:  1 ( 1.00 ) , delta:  tensor([0.0490], device='cuda:0', dtype=torch.float64)
pred:  1 ( 1.00 ) , delta:  tensor([0.0525], device='cuda:0', dtype=torch.float64)
pred:  1 ( 0.99 ) , delta:  tensor([0.0498], device='cuda:0', dtype=torch.float64)
pred:  0 ( 0.94 ) , delta:  tensor([0.5958], device='cuda:0', dtype=torch.float64)
pred:  1 ( 0.98 ) , delta:  tensor([0.0185], device='cuda:0', dtype=torch.float64)
pred:  0 ( 0.99 ) , delta:  tensor([0.6353], device='cuda:0', dtype=torch.float64)
pred:  0 ( 0.88 ) , delta:  tensor([0.5595], device='cuda:0', dtype=torch.float64)
pred:  1 ( 0.90 ) , delta:  tensor([0.0578], device='cuda:0', dtype=torch.float64)
pred:  1 ( 1.00 ) , delta:  tensor([0.0525], device='cuda:0', dtype=torch.float64)
pred:  1 ( 1.00 ) , delta:  tensor([0.0484], device='cuda:0', dtype=torch.float64)
pred:  1 ( 0.94 ) , delta:  tensor([0.0014], device='cuda:0', dtype=torch.float64)
pred:  1 ( 

[datetime.datetime(2012, 5, 9, 0, 0), datetime.datetime(2012, 5, 23, 0, 0)]
pred:  0 ( 1.00 ) , delta:  tensor([0.6357], device='cuda:0', dtype=torch.float64)


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
POSITIVE,NEGATIVE (1.00),POSITIVE,-2.75,f ##yi - - - - - - - - andrew gill ##um


[datetime.datetime(2013, 8, 28, 0, 0), datetime.datetime(2013, 9, 11, 0, 0)]
pred:  0 ( 0.97 ) , delta:  tensor([0.5982], device='cuda:0', dtype=torch.float64)
pred:  1 ( 0.93 ) , delta:  tensor([0.0168], device='cuda:0', dtype=torch.float64)
pred:  1 ( 0.80 ) , delta:  tensor([0.1030], device='cuda:0', dtype=torch.float64)


[datetime.datetime(2013, 9, 11, 0, 0), datetime.datetime(2013, 9, 25, 0, 0)]
pred:  0 ( 1.00 ) , delta:  tensor([0.6345], device='cuda:0', dtype=torch.float64)
pred:  0 ( 0.96 ) , delta:  tensor([0.6056], device='cuda:0', dtype=torch.float64)
pred:  0 ( 0.92 ) , delta:  tensor([0.5796], device='cuda:0', dtype=torch.float64)
